In [18]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class from your CRUD Python module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
db = AnimalShelter()

# Fetch all documents from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to avoid issues with DataTable
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load and encode the logo image
image_filename = 'Grazioso_Salvare_Logo.png'  # Make sure this file is in the same directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

# Create the logo component
logo = html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'100px', 'width':'auto'})

# Create your identifier
identifier = html.Div("Created by Jeff Conwi", style={'font-style': 'italic', 'padding': '10px'})

app.layout = html.Div([
    # Header
    html.Div([
        html.A([logo], href='https://www.snhu.edu'),  # Logo with link
        html.H1('Grazioso Salvare Dashboard'),
        identifier
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'space-between', 'padding': '10px'}),
    
    html.Hr(),
    
    # Filter options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    
    html.Hr(),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts
    html.Div(className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('graph-id', "children"),
     Output('map-id', "children")],
    [Input('filter-type', 'value'),
     Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_dashboard(filter_type, viewData, selected_rows):
    # Filter data based on filter_type
    if filter_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    results = list(db.read(query))
    df = pd.DataFrame.from_records(results)
    df.drop(columns=['_id'], inplace=True)
    data = df.to_dict('records')

    # Update graph
    if not viewData:
        graph = []
    else:
        dff = pd.DataFrame(viewData)
        breed_counts = dff['breed'].value_counts().nlargest(10)
        graph = [dcc.Graph(figure=px.pie(names=breed_counts.index, values=breed_counts.values, title='Top 10 Dog Breeds'))]

    # Update map
    if not viewData or selected_rows is None or len(selected_rows) == 0:
        map = [dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])]
    else:
        dff = pd.DataFrame(viewData)
        row = selected_rows[0]
        map = [
            dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                    dl.Tooltip(dff.iloc[row]['breed']),
                    dl.Popup([
                        html.H3("Animal Details"),
                        html.P(f"Name: {dff.iloc[row]['name']}"),
                        html.P(f"Animal Type: {dff.iloc[row]['animal_type']}"),
                        html.P(f"Breed: {dff.iloc[row]['breed']}"),
                        html.P(f"Color: {dff.iloc[row]['color']}"),
                        html.P(f"Age: {dff.iloc[row]['age_upon_outcome']}"),
                        html.P(f"Sex: {dff.iloc[row]['sex_upon_outcome']}")
                    ])
                ])
            ])
        ]

    return data, graph, map

# This callback will highlight selected rows in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': {'row_index': i},
        'background_color': '#D2F3FF'
    } for i in (selected_rows or [])]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:20051/
